# 1. Merge all files

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import pandas as pd

import selenium
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import os

import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [15]:
folder_path = "/Users/nguyentienhuy/Documents/huy/Project/xsmb/data_bds/"
file_list = os.listdir(folder_path)

merged_df = pd.DataFrame()

# Loop qua từng file CSV và merge vào DataFrame
for file in file_list:
    if "cho" in file:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        merged_df = pd.concat([merged_df, df], ignore_index=True)

print(len(merged_df))

30742


In [16]:
merged_df = merged_df.dropna()
merged_df.to_csv(f"data_bds/all_bds.csv", index=False, encoding="utf-8-sig")

# 2. Clean data 

In [137]:
df = pd.read_csv("data_bds/all_bds.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 30623


In [138]:
df = df[~df['Area'].str.contains(',')]
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 30371


2.0 Xử lý diện tích

In [139]:
# area_pattern = re.compile(r'(\d+) m²')
# df['area_value'] = df['Area'].apply(lambda x: int(area_pattern.search(x).group(1)) if area_pattern.search(x) else None)

df['area_value'] = df['Area'].apply(lambda x: float(x.replace(',', '').replace(' m²', '').replace('.', '')))


2.1 Xử lý price

In [140]:
# Tạo cột 'price_per_m2'
# 1. Price ghi: 70 nghìn / m2 => Cột 'price_per_m2' thành 70000
pattern = re.compile(r'(\d+) nghìn/m²')
df['price_per_m2'] = df['Price'].apply(lambda x: int(pattern.search(x).group(1)) if pattern.search(x) else None)

#2.Nếu Price ghi X triệu/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y
def convert_price_string_to_float(price_string):
    return float(price_string.replace(',', '.'))

price_pattern = re.compile(r'([\d,]+) triệu/tháng')

df['price_value_trieu'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern.search(x).group(1)) if price_pattern.search(x) else None)
df['price_per_m2_trieu'] = 1000000 * df['price_value_trieu'] / df['area_value']

#3.Nếu Price ghi X tỷ/tháng và Area ghi Y m2 thì => Cột 'price_per_m2' thành x / Y

price_pattern_bil = re.compile(r'([\d,]+) tỷ')

df['price_value_ty'] = df['Price'].apply(lambda x: convert_price_string_to_float(price_pattern_bil.search(x).group(1)) if price_pattern_bil.search(x) else None)
df['price_per_m2_ty'] = 1000000000 * df['price_value_ty'] / df['area_value']

#3. Nếu Price là thoả thuận thì bỏ qua khỏi df luôn
df = df[df['Price'] != 'Giá thỏa thuận']



In [141]:
#4. Từ các cột "price_per_m2", "price_per_m2_trieu", "price_per_m2_ty" ta tìm được price_per_m2_final

def calculate_price_per_m2_final(row):
    if not pd.isna(row['price_per_m2']):
        return row['price_per_m2']
    elif not pd.isna(row['price_per_m2_trieu']):
        return row['price_per_m2_trieu']
    elif not pd.isna(row['price_per_m2_ty']):
        return row['price_per_m2_ty']
    else:
        return np.nan

# Áp dụng hàm tùy chỉnh để tạo cột mới
df['price_per_m2_final'] = df.apply(calculate_price_per_m2_final, axis=1)

In [142]:
#5. Xử lý thành phố
df[['district', 'city']] = df['Location'].str.split(', ', expand=True)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28436 entries, 0 to 30622
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               28436 non-null  object 
 1   Price               28436 non-null  object 
 2   Area                28436 non-null  object 
 3   Location            28436 non-null  object 
 4   type                28436 non-null  object 
 5   area_value          28436 non-null  float64
 6   price_per_m2        3733 non-null   float64
 7   price_value_trieu   24522 non-null  float64
 8   price_per_m2_trieu  24522 non-null  float64
 9   price_value_ty      38 non-null     float64
 10  price_per_m2_ty     38 non-null     float64
 11  price_per_m2_final  28293 non-null  float64
 12  district            28436 non-null  object 
 13  city                28436 non-null  object 
dtypes: float64(7), object(7)
memory usage: 3.3+ MB


In [144]:
df.to_csv(f"data_bds/all_bds_clean.csv", index=False, encoding="utf-8-sig")

# 3. Model - test

In [170]:
df = pd.read_csv("data_bds/all_bds_clean.csv")
df.head()
print(f"The total rows of dataframe is {len(df)}")

The total rows of dataframe is 28436


In [171]:
df['district'] = df['district'].replace('·\nQuận', 'Quận', regex=True)
df['district'] = df['district'].replace('·\nTây Hồ', 'Tây Hồ', regex=True)
df['district'] = df['district'].replace('·\nCầu Giấy', 'Cầu Giấy', regex=True)
df['district'] = df['district'].replace('·\nBa Đình', 'Ba Đình', regex=True)
df['district'] = df['district'].replace('·\nHà Đông', 'Hà Đông', regex=True)
df['district'] = df['district'].replace('·\nNam Từ Liêm', 'Nam Từ Liêm', regex=True)


In [159]:
df['type'].unique()

array(['cho-thue-nha-tro-phong-tro', 'cho-thue-kho-nha-xuong-dat',
       'cho-thue-van-phong', 'cho-thue-sang-nhuong-cua-hang-ki-ot',
       'cho-thue-loai-bat-dong-san-khac'], dtype=object)

In [160]:
replace_dict = {
    'cho-thue-nha-tro-phong-tro': 'Cho thuê nhà trọ phòng trọ',
    'cho-thue-kho-nha-xuong-dat': 'Cho thuê kho nhà xưởng',
    'cho-thue-van-phong': 'Cho thuê văn phòng',
    'cho-thue-sang-nhuong-cua-hang-ki-ot': 'Cho thuê sang nhượng cửa hàng kiot',
    'cho-thue-loai-bat-dong-san-khac': 'Cho thuê loại bất động sản khác'
}

# Thay đổi giá trị trong cột 'type' sử dụng replace
df['type'] = df['type'].replace(replace_dict)

In [162]:
print(df.columns)
df = df[["area_value", "price_per_m2_final", "district", "city", "type"]]

Index(['Title', 'Price', 'Area', 'Location', 'type', 'area_value',
       'price_per_m2', 'price_value_trieu', 'price_per_m2_trieu',
       'price_value_ty', 'price_per_m2_ty', 'price_per_m2_final', 'district',
       'city'],
      dtype='object')


In [164]:
df = df.dropna(subset=['price_per_m2_final'])
df = df[(df['price_per_m2_final'] > 20000) & (df['price_per_m2_final'] < 5000000)]

In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df.drop('price_per_m2_final', axis=1)
y = df['price_per_m2_final']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_features = ['area_value']
categorical_features = ['district', 'city', 'type']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Xây dựng pipeline với mô hình Linear Regression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f'R^2 Score: {score}')

R^2 Score: 0.3143662208441329


In [168]:
import pickle

# Lưu mô hình vào file
model_filename_pickle = 'model/linear_regression_model.pkl'
with open(model_filename_pickle, 'wb') as file:
    pickle.dump(model, file)

# Để sử dụng mô hình từ file pickle
with open(model_filename_pickle, 'rb') as file:
    loaded_model_pickle = pickle.load(file)


In [169]:
new_data_1 = {'area_value': [25],
            'district': ['Thanh Xuân'],
            'city': ['Hà Nội'],
            'type': ['Cho thuê nhà trọ phòng trọ']}

new_df_1 = pd.DataFrame(new_data_1)

new_prediction_pickle = loaded_model_pickle.predict(new_df_1)
print(f'Predicted Price per m2 using loaded pickle model: {new_prediction_pickle}')

Predicted Price per m2 using loaded pickle model: [140201.03068474]
